# **Import the necessary libraries**

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout


# **Apply data augmentation technics and load the images**

In [28]:
train_dir = "/kaggle/input/data-cnn/data_cnn/train"
val_dir = "/kaggle/input/data-cnn/data_cnn/val"
test_dir = "/kaggle/input/data-cnn/data_cnn/test"


train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)
val_test_datagen = ImageDataGenerator(rescale=1./255,rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
val_generator = val_test_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
test_generator = val_test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)


Found 7602 images belonging to 2 classes.
Found 1628 images belonging to 2 classes.
Found 1632 images belonging to 2 classes.


In [29]:
print("Train classes:", train_generator.class_indices)
print("Validation classes:", val_generator.class_indices)
print("Test classes:", test_generator.class_indices)


Train classes: {'safe': 0, 'unsafe': 1}
Validation classes: {'safe': 0, 'unsafe': 1}
Test classes: {'safe': 0, 'unsafe': 1}


# **Build Model**

In [32]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze pretrained layers


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,410,532 (16.82 MB)

 Trainable params: 360,961 (1.38 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [34]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)



# **Train Model**

In [35]:

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # Increased epochs, but will stop early if overfitting
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 152s 554ms/step - accuracy: 0.4868 - loss: 0.6954 - val_accuracy: 0.5184 - val_loss: 0.6925 - learning_rate: 1.0000e-04
Epoch 2/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 119s 489ms/step - accuracy: 0.4874 - loss: 0.6942 - val_accuracy: 0.5184 - val_loss: 0.6932 - learning_rate: 1.0000e-04
Epoch 3/10
237/238 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step - accuracy: 0.5127 - loss: 0.6930
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
238/238 ━━━━━━━━━━━━━━━━━━━━ 120s 493ms/step - accuracy: 0.5128 - loss: 0.6930 - val_accuracy: 0.5184 - val_loss: 0.6925 - learning_rate: 1.0000e-04
Epoch 4/10
238/238 ━━━━━━━━━━━━━━━━━━━━ 119s 489ms/step - accuracy: 0.5145 - loss: 0.6933 - val_accuracy: 0.5184 - val_loss: 0.6925 - learning_rate: 5.0000e-05


# **Unfreeze last 30 layers and finetune the model acc to our custom dataset**

In [36]:

for layer in base_model.layers[:-30]:  
    layer.trainable = False

# Recompile with a very low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Use Early Stopping & Learning Rate Scheduler
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

# Fine-tune the model
history_fine = model.fit(train_generator, validation_data=val_generator, 
                         epochs=20, callbacks=[early_stopping, lr_scheduler])


Epoch 1/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 151s 549ms/step - accuracy: 0.5019 - loss: 0.6934 - val_accuracy: 0.5184 - val_loss: 0.6925 - learning_rate: 1.0000e-05
Epoch 2/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 120s 491ms/step - accuracy: 0.5227 - loss: 0.6922 - val_accuracy: 0.5184 - val_loss: 0.6925 - learning_rate: 1.0000e-05
Epoch 3/20
237/238 ━━━━━━━━━━━━━━━━━━━━ 0s 406ms/step - accuracy: 0.5140 - loss: 0.6924
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
238/238 ━━━━━━━━━━━━━━━━━━━━ 120s 490ms/step - accuracy: 0.5139 - loss: 0.6924 - val_accuracy: 0.5184 - val_loss: 0.6925 - learning_rate: 1.0000e-05
Epoch 4/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 120s 491ms/step - accuracy: 0.5065 - loss: 0.6926 - val_accuracy: 0.5184 - val_loss: 0.6925 - learning_rate: 5.0000e-06


# **Test on the test set**

In [37]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")


51/51 ━━━━━━━━━━━━━━━━━━━━ 23s 461ms/step - accuracy: 0.1783 - loss: 0.7231
Test Accuracy: 51.84%


In [38]:
model.save("/kaggle/working/safe_unsafe_efficientnet.h5")


In [48]:
from tensorflow.keras.models import load_model

model = load_model("/kaggle/working/safe_unsafe_efficientnet.h5")


# **Test the model on a completely new unseen dataset to see if the model generalizes well on images**

In [49]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

new_data_gen = ImageDataGenerator(rescale=1./255)  # Ensure same preprocessing

new_data = new_data_gen.flow_from_directory(
    "/kaggle/input/cnn-data-test/Child Safety.v2i.folder (1)/valid",  
    target_size=(224, 224),  # Use same size as training
    batch_size=32, 
    class_mode="binary"  # Use same class mode as training
)


Found 512 images belonging to 2 classes.


In [50]:
test_loss, test_acc = model.evaluate(new_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 115ms/step - accuracy: 0.5144 - loss: 0.6928
Test Accuracy: 50.39%
